In [4]:
# !pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
# !pip install -q datasets bitsandbytes einops wandb

In [28]:

!pip install bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### load dataset

In [1]:
import json
def formatting_func(example):
    text = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
    return text

f = open('kbase_all.json') 
data = json.load(f) 
res = []
for d in data['data']:
    res.append({'text': formatting_func(d)})
d = {"data": res}
with open('kbase_formatted_data.json', 'w') as f:
    json.dump(d, f)

In [1]:
from datasets import load_dataset
from datasets import load_dataset,Features,Value
# dataset = load_dataset("json",data_files="kbase_formatted_data.json", split="train", field='data')
dataset = load_dataset("json",data_files="kbase_formatted_data.json", split="train", field='data')

dataset

Dataset({
    features: ['text'],
    num_rows: 3040
})

### load model

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b-instruct"
# model_name = "fine_tuned_model_kbase_data"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

### model fine tune

In [4]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [9]:
from transformers import TrainingArguments

output_dir = "./ziming_results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4 #4*4=16
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 2
learning_rate = 2e-4 # if 4 gpu, decrease the lr
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [10]:
from trl import SFTTrainer
# max_seq_length = 512
max_seq_length = 215

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [11]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [12]:
%env WANDB_CONFIG_DIR="/home/ac.zyang/wandb"
trainer.train()

env: WANDB_CONFIG_DIR="/home/ac.zyang/wandb"


/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
2,1.800100
4,1.917100
6,1.825200
8,1.788800
10,1.601800
12,1.584400
14,1.624200
16,1.637100
18,1.554200
20,1.499900


/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarnin

/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarnin

TrainOutput(global_step=500, training_loss=0.6539567287564277, metrics={'train_runtime': 12698.1343, 'train_samples_per_second': 2.52, 'train_steps_per_second': 0.039, 'total_flos': 1.449998331777024e+17, 'train_loss': 0.6539567287564277, 'epoch': 10.53})

In [13]:
trainer.save_model('fine_tuned_model_kbase_all_data')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
tuned_model = AutoModelForCausalLM.from_pretrained('fine_tuned_model_kbase_data')

In [3]:
tuned_model = tuned_model.eval()
 
tokenizer = AutoTokenizer.from_pretrained('fine_tuned_model_kbase_data')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### answer after finetuning

In [4]:
prompt = """
The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.
 
Current conversation:
 
Human: 'What file formats can be used to upload FBA models into KBase??
AI:
""".strip()
 
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(tuned_model.device)

In [ ]:
with torch.inference_mode():
    outputs = tuned_model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

In [ ]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

### standard answer

In [ ]:
# answer in training dataset
standard_answer = """
 "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. 
 In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. 
 The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.
 Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). 
 In these industries, workers often face low wages, limited benefits, and reduced bargaining power, 
 leading to a situation where they are dependent on the employer for their livelihood. This dependence can result 
 in further suppression of wages and a decline in working conditions.\n\nOverall, the concept of monopsony is 
 essential to understanding the dynamics of labor markets and the impact of market power on workers. 
 Further research is needed to understand the extent and impact of monopsonies on the economy and to develop 
 policies to address this issue.\n\nReferences:\nBivens, J., & Mishel, L. (2013). The Pay of Corporate Executives 
 and Financial Professionals as Evidence of Rents in Top 1 Percent Incomes. Journal of Economic Perspectives, 
 27(3), 57-78.
"""
standard_answer